# Data Gathering
This notebook contains the logic for gathering datasets for the project. Data is gathered from the AlphaVantage API where the rate limiting is 15 requests per minute. Helper functions are provided to specify date ranges for specific assets to download data while automatically avoiding hitting the rate limit.

**_getStockData_**: Gets data for a specific asset for a specific month and specific time interval (1 hour etc).

**_getDataforPeriod_**: Reuses the _getStockData_ function multiple times over different months to gather data for a range of specified years. (2018 - 2020).

In [1]:
import requests
import pandas as pd
import numpy as np
import time

# Helper function that fetches the data.
def getStockData(API_KEY, symbol, function, market=None, outputsize="full", interval=None, month=None, count=0):
    symbolString = ""
    intervalString = ""
    outputsizeString = ""
    monthString = ""
    marketString = ""

    count = checkRequests(count)
    
    if symbol != None:
        symbolString = f"&symbol={symbol}" 
    if interval != None:
        intervalString = f"&interval={interval}" 
    if outputsize != None:
        outputsizeString = f"&outputsize={outputsize}" 
    if month != None:
        monthString = f"&month={month}"
    if market != None:
        marketString = f"&market={market}"

    URL = f"https://www.alphavantage.co/query?function={function}{symbolString}{marketString}{intervalString}{outputsizeString}{monthString}&apikey={API_KEY}"
    response = requests.get(URL)
    data = response.json()
    if data == None:
        print("None")
    data = data.get(f"Time Series ({interval})", {})
    df = pd.DataFrame.from_dict(data, orient="index")
    df.insert(0, "timestamp", df.index)
    return df, count

# Gets stock data for a a period of years specified by starting year and ending year.
def getDataForPeriod(API_KEY, symbol, function, interval, yearStart, yearEnd, count, market=None):
    dfList = []
    for year in range(yearStart, yearEnd+1):
        print(f"Getting {symbol} data for {year}")
        for month in range(1, 13):
            month = f"{year}-{str(month).zfill(2)}"
            df, count = getStockData(API_KEY=API_KEY, symbol=symbol, function=function, interval=interval, month=month, count=count, market=market)
            dfList.append(df)
    full_df = pd.concat(dfList, ignore_index=True)
    return full_df, count

# Checks if 75 requests has been made and sets a 1 minute timer before requesting again.
def checkRequests(count):
    if count == 75:
        print("API requests reached for min... Waiting")
        time.sleep(62)
        count = 0
    count += 1
    return count

### Stock Data Gathering

Both **Hourly** and **Daily** data is gathered for 3 assets (**AAPL**, **TSLA**, **AMZN**).

We will calculate the data for **4-Hour** and **Weekly** timeframes for these assets by resampling the data gathered into larger timeframes.

In [2]:
API_KEY = "5HUC90FRQ4H9PK0Q"

# Gets Stock Data 1hr and 1 day, 4hr can be generated from this data. Gets weekly via CSV dataset files downloaded from Kaggle.
count = 0
aapl_data_1hr, count = getDataForPeriod(API_KEY=API_KEY, symbol="AAPL", function="TIME_SERIES_INTRADAY", interval="60min", yearStart=2018, yearEnd=2024, count=count)
tsla_data_1hr, count = getDataForPeriod(API_KEY=API_KEY, symbol="TSLA", function="TIME_SERIES_INTRADAY", interval="60min", yearStart=2018, yearEnd=2024, count=count)
amzn_data_1hr, count = getDataForPeriod(API_KEY=API_KEY, symbol="AMZN", function="TIME_SERIES_INTRADAY", interval="60min", yearStart=2018, yearEnd=2024, count=count)

aapl_data_daily, count = getStockData(API_KEY=API_KEY, symbol="AAPL", function="TIME_SERIES_DAILY", outputsize="full", interval="Daily", count=count)
tsla_data_daily, count = getStockData(API_KEY=API_KEY, symbol="TSLA", function="TIME_SERIES_DAILY", outputsize="full", interval="Daily", count=count)
amzn_data_daily, count = getStockData(API_KEY=API_KEY, symbol="AMZN", function="TIME_SERIES_DAILY", outputsize="full", interval="Daily", count=count)

Getting AAPL data for 2018
Getting AAPL data for 2019
Getting AAPL data for 2020
Getting AAPL data for 2021
Getting AAPL data for 2022
Getting AAPL data for 2023
Getting AAPL data for 2024
API requests reached for min... Waiting
Getting TSLA data for 2018
Getting TSLA data for 2019
Getting TSLA data for 2020
Getting TSLA data for 2021
Getting TSLA data for 2022
Getting TSLA data for 2023
API requests reached for min... Waiting
Getting TSLA data for 2024
Getting AMZN data for 2018
Getting AMZN data for 2019
Getting AMZN data for 2020
Getting AMZN data for 2021
Getting AMZN data for 2022
API requests reached for min... Waiting
Getting AMZN data for 2023
Getting AMZN data for 2024


### Data Checking

We first quickly check for missing values in our datasets as this will cause inaccuracte data when resampling.

In [3]:
aapl_data_1hr.head()

,timestamp,1. open,2. high,3. low,4. close,5. volume
0,2018-01-31 19:00:00,39.1727,39.2103,39.1022,39.2103,283016
1,2018-01-31 18:00:00,39.2291,39.2902,39.0857,39.1633,253676
2,2018-01-31 17:00:00,39.1680,39.2878,39.1374,39.2197,383444
3,2018-01-31 16:00:00,39.3536,39.3607,39.1257,39.1374,13960772
4,2018-01-31 15:00:00,39.2173,39.3971,39.1844,39.3536,21420456


In [4]:
aapl_data_1hr.isna().any()

timestamp    False
1. open      False
2. high      False
3. low       False
4. close     False
5. volume    False
dtype: bool

In [5]:
tsla_data_1hr.head()

,timestamp,1. open,2. high,3. low,4. close,5. volume
0,2018-01-31 19:00:00,23.6400,23.7013,23.6400,23.7000,106980
1,2018-01-31 18:00:00,23.6200,23.6600,23.6133,23.6207,36000
2,2018-01-31 17:00:00,23.5667,23.6273,23.5567,23.6273,22500
3,2018-01-31 16:00:00,23.6207,23.6407,23.5833,23.6000,11346030
4,2018-01-31 15:00:00,23.5413,23.6647,23.4000,23.6307,9568875


In [6]:
tsla_data_1hr.isna().any()

timestamp    False
1. open      False
2. high      False
3. low       False
4. close     False
5. volume    False
dtype: bool

In [7]:
amzn_data_1hr.head()

,timestamp,1. open,2. high,3. low,4. close,5. volume
0,2018-01-31 19:00:00,73.0500,73.1500,73.0185,73.1500,26060
1,2018-01-31 18:00:00,72.9500,73.0840,72.9000,72.9500,60020
2,2018-01-31 17:00:00,72.3585,73.0525,72.3500,72.9845,351100
3,2018-01-31 16:00:00,72.5775,72.7000,72.2500,72.5445,6935520
4,2018-01-31 15:00:00,73.1650,73.2215,72.5020,72.5695,22019040


In [8]:
amzn_data_1hr.isna().any()

timestamp    False
1. open      False
2. high      False
3. low       False
4. close     False
5. volume    False
dtype: bool

In [9]:
aapl_data_daily.head()

,timestamp,1. open,2. high,3. low,4. close,5. volume
2025-04-04,2025-04-04,193.8900,199.8800,187.3400,188.3800,125910913
2025-04-03,2025-04-03,205.5400,207.4900,201.2500,203.1900,103419006
2025-04-02,2025-04-02,221.3150,225.1900,221.0200,223.8900,35905904
2025-04-01,2025-04-01,219.8050,223.6800,218.9000,223.1900,36412740
2025-03-31,2025-03-31,217.0050,225.6200,216.2300,222.1300,65299321


In [10]:
aapl_data_daily.isna().any()

timestamp    False
1. open      False
2. high      False
3. low       False
4. close     False
5. volume    False
dtype: bool

In [11]:
tsla_data_daily.head()

,timestamp,1. open,2. high,3. low,4. close,5. volume
2025-04-04,2025-04-04,255.3800,261.0000,236.0000,239.4300,181229353
2025-04-03,2025-04-03,265.2900,276.3000,261.5100,267.2800,136174291
2025-04-02,2025-04-02,254.6000,284.9900,251.2700,282.7600,212787817
2025-04-01,2025-04-01,263.8000,277.4500,259.2500,268.4600,146486911
2025-03-31,2025-03-31,249.3100,260.5600,243.3601,259.1600,134008936


In [12]:
tsla_data_daily.isna().any()

timestamp    False
1. open      False
2. high      False
3. low       False
4. close     False
5. volume    False
dtype: bool

In [13]:
amzn_data_daily.head()

,timestamp,1. open,2. high,3. low,4. close,5. volume
2025-04-04,2025-04-04,167.1450,178.1436,166.0000,171.0000,123159359
2025-04-03,2025-04-03,182.9950,184.1300,176.9200,178.4100,95553617
2025-04-02,2025-04-02,187.6600,198.3400,187.6600,196.0100,53679198
2025-04-01,2025-04-01,187.8600,193.9300,187.2000,192.1700,41267315
2025-03-31,2025-03-31,188.1900,191.3300,184.4000,190.2600,63547558


In [14]:
amzn_data_daily.isna().any()

timestamp    False
1. open      False
2. high      False
3. low       False
4. close     False
5. volume    False
dtype: bool

### Data Resampling

By default after gathering data, numerical data are of the type objects, we will convert this for each dataset to the correct data type and then proceed with resampling the data into larger timeframes.

Data for **ETH** and **BTC** were gathered from Kaggle in **Hourly** timeframes. The CSV file was manually placed in the datasets directory. We will read it from there and perform resampling.

In [15]:
aapl_data_1hr.dtypes

timestamp    object
1. open      object
2. high      object
3. low       object
4. close     object
5. volume    object
dtype: object

In [16]:
aapl_data_1hr["timestamp"] = pd.to_datetime(aapl_data_1hr["timestamp"])
aapl_data_1hr.set_index("timestamp", inplace=True)
cols = ["1. open", "2. high", "3. low", "4. close", "5. volume"]
aapl_data_1hr[cols] = aapl_data_1hr[cols].apply(pd.to_numeric)

aapl_data_4hr = aapl_data_1hr.resample("4h").agg({
    "1. open": "first",
    "2. high": "max",
    "3. low": "min",
    "4. close": "last",
    "5. volume": "sum"
}).dropna()

aapl_data_4hr.reset_index(inplace=True)

aapl_data_4hr.head()

,timestamp,1. open,2. high,3. low,4. close,5. volume
0,2018-01-02 04:00:00,39.7366,39.9951,39.7131,39.9599,233140
1,2018-01-02 08:00:00,39.9505,40.3241,39.7742,40.2325,46810976
2,2018-01-02 12:00:00,40.2313,40.4886,40.2278,40.4792,36005084
3,2018-01-02 16:00:00,40.4792,40.4886,40.4228,40.4886,9113536
4,2018-01-03 04:00:00,40.5732,40.6038,40.5356,40.5474,118400


In [17]:
tsla_data_1hr["timestamp"] = pd.to_datetime(tsla_data_1hr["timestamp"])
tsla_data_1hr.set_index("timestamp", inplace=True)
cols = ["1. open", "2. high", "3. low", "4. close", "5. volume"]
tsla_data_1hr[cols] = tsla_data_1hr[cols].apply(pd.to_numeric)
tsla_data_1hr.dtypes

tsla_data_4hr = tsla_data_1hr.resample("4h").agg({
    "1. open": "first",
    "2. high": "max",
    "3. low": "min",
    "4. close": "last",
    "5. volume": "sum"
}).dropna()

tsla_data_4hr.reset_index(inplace=True)

tsla_data_4hr.head()

,timestamp,1. open,2. high,3. low,4. close,5. volume
0,2018-01-02 04:00:00,20.8000,20.800,20.7167,20.7333,173010
1,2018-01-02 08:00:00,20.7667,21.474,20.7327,21.3677,31475715
2,2018-01-02 12:00:00,21.3677,21.438,21.2560,21.3527,18547800
3,2018-01-02 16:00:00,21.3687,21.400,21.3587,21.3700,1313325
4,2018-01-03 04:00:00,21.4333,21.450,21.4000,21.4013,77115


In [18]:
amzn_data_1hr["timestamp"] = pd.to_datetime(amzn_data_1hr["timestamp"])
amzn_data_1hr.set_index("timestamp", inplace=True)
cols = ["1. open", "2. high", "3. low", "4. close", "5. volume"]
amzn_data_1hr[cols] = amzn_data_1hr[cols].apply(pd.to_numeric)
amzn_data_1hr.dtypes

amzn_data_4hr = amzn_data_1hr.resample("4h").agg({
    "1. open": "first",
    "2. high": "max",
    "3. low": "min",
    "4. close": "last",
    "5. volume": "sum"
}).dropna()

amzn_data_4hr.reset_index(inplace=True)

amzn_data_4hr.head()

,timestamp,1. open,2. high,3. low,4. close,5. volume
0,2018-01-02 04:00:00,58.3900,58.7105,58.3900,58.6945,44700
1,2018-01-02 08:00:00,58.6945,59.3960,58.4735,59.3655,17878240
2,2018-01-02 12:00:00,59.3605,59.5000,59.2565,59.4500,14718120
3,2018-01-02 16:00:00,59.4505,59.4505,59.2000,59.3000,3563960
4,2018-01-03 04:00:00,59.4385,59.4385,59.3450,59.3500,20720


In [19]:
eth_data_1hr = pd.read_csv("datasets/crypto/hourly/ETH.csv")
eth_data_1hr.head()

,Date,Symbol,Open,High,Low,Close,Volume ETH,Volume USDT,tradecount
0,2023-10-19 23:00:00,ETHUSDT,1564.22,1567.31,1563.70,1566.57,3403.8235,5.327189e+06,12103
1,2023-10-19 22:00:00,ETHUSDT,1563.45,1568.00,1563.39,1564.22,4833.0579,7.566292e+06,12073
2,2023-10-19 21:00:00,ETHUSDT,1566.51,1567.72,1563.39,1563.46,5240.2983,8.200644e+06,15157
3,2023-10-19 20:00:00,ETHUSDT,1568.00,1573.15,1563.72,1566.51,8725.2965,1.369178e+07,19124
4,2023-10-19 19:00:00,ETHUSDT,1561.66,1569.92,1561.20,1568.00,6092.9619,9.540306e+06,16003


In [20]:
eth_data_1hr.isna().any()

Date           False
Symbol         False
Open           False
High           False
Low            False
Close          False
Volume ETH     False
Volume USDT    False
tradecount     False
dtype: bool

In [21]:
eth_data_1hr.dtypes

Date            object
Symbol          object
Open           float64
High           float64
Low            float64
Close          float64
Volume ETH     float64
Volume USDT    float64
tradecount       int64
dtype: object

In [22]:
eth_data_1hr["Date"] = pd.to_datetime(eth_data_1hr["Date"], format='mixed')  # Convert to datetime
eth_data_1hr.set_index("Date", inplace=True)


cols = ["Open", "High", "Low", "Close", "Volume ETH", "Volume USDT", "tradecount"]
eth_data_1hr[cols] = eth_data_1hr[cols].apply(pd.to_numeric)
eth_data_1hr.dtypes

eth_data_4hr = eth_data_1hr.resample("4h").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume ETH": "sum",
    "Volume USDT": "sum",
    "tradecount": "sum"
}).dropna()

eth_data_daily = eth_data_1hr.resample("D").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume ETH": "sum",
    "Volume USDT": "sum",
    "tradecount": "sum"
}).dropna()

eth_data_weekly = eth_data_1hr.resample("W").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume ETH": "sum",
    "Volume USDT": "sum",
    "tradecount": "sum"
}).dropna()

In [23]:
eth_data_1hr.head()

,Symbol,Open,High,Low,Close,Volume ETH,Volume USDT,tradecount
Date,,,,,,,,
2023-10-19 23:00:00,ETHUSDT,1564.22,1567.31,1563.70,1566.57,3403.8235,5.327189e+06,12103
2023-10-19 22:00:00,ETHUSDT,1563.45,1568.00,1563.39,1564.22,4833.0579,7.566292e+06,12073
2023-10-19 21:00:00,ETHUSDT,1566.51,1567.72,1563.39,1563.46,5240.2983,8.200644e+06,15157
2023-10-19 20:00:00,ETHUSDT,1568.00,1573.15,1563.72,1566.51,8725.2965,1.369178e+07,19124
2023-10-19 19:00:00,ETHUSDT,1561.66,1569.92,1561.20,1568.00,6092.9619,9.540306e+06,16003


In [24]:
eth_data_4hr.head()

,Open,High,Low,Close,Volume ETH,Volume USDT,tradecount
Date,,,,,,,
2017-08-17 04:00:00,301.13,307.96,298.00,307.96,1561.95305,473487.665119,711
2017-08-17 08:00:00,307.95,312.00,307.00,308.95,1177.71088,364545.316402,775
2017-08-17 12:00:00,308.95,310.51,303.56,307.06,1882.05267,578644.931890,1140
2017-08-17 16:00:00,307.74,312.18,298.21,301.60,1208.05192,370209.051467,957
2017-08-17 20:00:00,301.60,310.85,299.01,302.00,1200.94182,367768.335479,939


In [25]:
eth_data_weekly.head()

,Open,High,Low,Close,Volume ETH,Volume USDT,tradecount
Date,,,,,,,
2017-08-20,301.13,312.18,278.00,299.10,21224.89324,6.376472e+06,14013
2017-08-27,299.10,348.13,144.21,348.13,45233.88589,1.456741e+07,42362
2017-09-03,348.11,394.39,320.08,341.77,33886.41427,1.229073e+07,35671
2017-09-10,339.29,344.55,235.55,293.50,35485.23332,1.071100e+07,33803
2017-09-17,293.51,316.46,192.00,257.55,58259.63439,1.499828e+07,44796


In [26]:
btc_data_1hr = pd.read_csv("datasets/crypto/hourly/BTC.csv")
btc_data_1hr.head()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
0,2023-10-19 23:00:00,BTCUSDT,28649.64,28717.82,28630.19,28713.71,613.28448,1.758298e+07,27099
1,2023-10-19 22:00:00,BTCUSDT,28604.09,28704.99,28598.60,28649.65,767.85518,2.200420e+07,28479
2,2023-10-19 21:00:00,BTCUSDT,28715.39,28739.73,28604.09,28604.09,1118.51697,3.205666e+07,42623
3,2023-10-19 20:00:00,BTCUSDT,28750.75,28882.07,28684.80,28715.38,1737.14711,4.997623e+07,59830
4,2023-10-19 19:00:00,BTCUSDT,28646.98,28765.76,28645.00,28750.75,1039.20935,2.984059e+07,49985


In [27]:
btc_data_1hr.dtypes

Date            object
Symbol          object
Open           float64
High           float64
Low            float64
Close          float64
Volume BTC     float64
Volume USDT    float64
tradecount       int64
dtype: object

In [28]:
btc_data_1hr.isna().any()

Date           False
Symbol         False
Open           False
High           False
Low            False
Close          False
Volume BTC     False
Volume USDT    False
tradecount     False
dtype: bool

In [29]:
btc_data_1hr["Date"] = btc_data_1hr["Date"].str.split(".").str[0]  # Remove milliseconds
btc_data_1hr["Date"] = pd.to_datetime(btc_data_1hr["Date"], errors="coerce")  # Convert to datetime

btc_data_1hr.set_index("Date", inplace=True)
cols = ["Open", "High", "Low", "Close", "Volume BTC", "Volume USDT", "tradecount"]
btc_data_1hr[cols] = btc_data_1hr[cols].apply(pd.to_numeric)
btc_data_1hr.dtypes

btc_data_4hr = btc_data_1hr.resample("4h").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume BTC": "sum",
    "Volume USDT": "sum",
    "tradecount": "sum"
}).dropna()

btc_data_daily = btc_data_1hr.resample("D").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume BTC": "sum",
    "Volume USDT": "sum",
    "tradecount": "sum"
}).dropna()

btc_data_weekly = btc_data_1hr.resample("W").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume BTC": "sum",
    "Volume USDT": "sum",
    "tradecount": "sum"
}).dropna()

In [30]:
btc_data_4hr.head()

,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
Date,,,,,,,
2017-08-17 04:00:00,4261.48,4349.99,4261.32,4349.99,82.088865,3.531943e+05,334
2017-08-17 08:00:00,4333.32,4485.39,4333.32,4427.30,63.619882,2.825012e+05,248
2017-08-17 12:00:00,4436.06,4485.39,4333.42,4352.34,174.562001,7.742388e+05,858
2017-08-17 16:00:00,4352.33,4354.84,4200.74,4325.23,225.109716,9.652911e+05,986
2017-08-17 20:00:00,4307.56,4369.69,4258.56,4285.08,249.769913,1.079545e+06,1001


In [31]:
btc_data_daily.head()

,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
Date,,,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,3.454770e+06,3427
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,5.086958e+06,5233
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972


In [32]:
btc_data_weekly.head()

,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
Date,,,,,,,
2017-08-20,4261.48,4485.39,3850.00,4086.29,2843.431426,1.202158e+07,13134
2017-08-27,4069.13,4453.91,3400.00,4310.01,4599.396629,1.897898e+07,37826
2017-09-03,4310.01,4939.19,4124.54,4509.08,4753.843376,2.158168e+07,35717
2017-09-10,4505.00,4788.59,3603.00,4130.37,6382.787745,2.730782e+07,42372
2017-09-17,4153.62,4394.59,2817.00,3699.99,8106.705127,3.002680e+07,50423


### Store Datasets

All the datasets either loaded from APIs or generated with resampling are now saved to the datasets directory.

In [33]:
import os

directories = ["./datasets/stocks/hourly", "./datasets/stocks/4hourly", "./datasets/stocks/daily", "./datasets/stocks/weekly", "./datasets/crypto/hourly", "./datasets/crypto/4hourly", "./datasets/crypto/daily", "./datasets/crypto/weekly"]
for dir in directories:
    os.makedirs(dir, exist_ok=True)

aapl_data_1hr.to_csv("datasets/stocks/hourly/AAPL.csv")
aapl_data_4hr.to_csv("datasets/stocks/4hourly/AAPL.csv", index=False)
aapl_data_daily.to_csv("datasets/stocks/daily/AAPL.csv", index=False)

tsla_data_1hr.to_csv("datasets/stocks/hourly/TSLA.csv")
tsla_data_4hr.to_csv("datasets/stocks/4hourly/TSLA.csv")
tsla_data_daily.to_csv("datasets/stocks/daily/TSLA.csv")

amzn_data_1hr.to_csv("datasets/stocks/hourly/AMZN.csv")
amzn_data_4hr.to_csv("datasets/stocks/4hourly/AMZN.csv")
amzn_data_daily.to_csv("datasets/stocks/daily/AMZN.csv")

eth_data_4hr.to_csv("datasets/crypto/4hourly/ETH.csv")
eth_data_daily.to_csv("datasets/crypto/daily/ETH.csv")
eth_data_weekly.to_csv("Datasets/crypto/weekly/ETH.csv")

btc_data_4hr.to_csv("datasets/crypto/4hourly/BTC.csv")
btc_data_daily.to_csv("datasets/crypto/daily/BTC.csv")
btc_data_weekly.to_csv("datasets/crypto/weekly/BTC.csv")